# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from random import sample

import re

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [5]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,...,0,2,0,4,1,1,0,1,1,0
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,...,1,3,1,0,0,1,2,0,0,0
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,3,0,2,1,1,0,1,0,1,0
1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,3,9,3,3,...,3,1,1,2,0,2,2,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12191,FOODS_3_823_CA_4_validation,FOODS_3_823,FOODS_3,FOODS,CA_4,CA,1,1,1,4,...,3,1,2,2,0,1,1,3,3,0
12192,FOODS_3_824_CA_4_validation,FOODS_3_824,FOODS_3,FOODS,CA_4,CA,0,0,0,1,...,0,1,1,1,0,2,0,0,0,1
12193,FOODS_3_825_CA_4_validation,FOODS_3_825,FOODS_3,FOODS,CA_4,CA,2,3,2,1,...,0,2,0,1,3,1,1,0,2,0
12194,FOODS_3_826_CA_4_validation,FOODS_3_826,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,1,2,2,1,4,5,1,7,0,4


In [6]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Foods'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [7]:
len(sales_intermittent)

1301

In [8]:
len(sales_lumpy)

395

In [9]:
len(sales_erratic)

458

In [10]:
len(sales_smooth)

3594

### Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)

In [11]:
sales_intermittent

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1618,FOODS_1_008_CA_1_validation,FOODS_1_008,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1619,FOODS_1_009_CA_1_validation,FOODS_1_009,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1620,FOODS_1_010_CA_1_validation,FOODS_1_010,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
1621,FOODS_1_011_CA_1_validation,FOODS_1_011,FOODS_1,FOODS,CA_1,CA,2,1,0,0,...,0,2,0,1,3,2,3,1,0,1
1627,FOODS_1_017_CA_1_validation,FOODS_1_017,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,2,2,0,1,0,0,1,2,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12174,FOODS_3_806_CA_4_validation,FOODS_3_806,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
12189,FOODS_3_821_CA_4_validation,FOODS_3_821,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,0,0,0,1,1,0,3,0,0
12191,FOODS_3_823_CA_4_validation,FOODS_3_823,FOODS_3,FOODS,CA_4,CA,1,1,1,4,...,3,1,2,2,0,1,1,3,3,0
12192,FOODS_3_824_CA_4_validation,FOODS_3_824,FOODS_3,FOODS,CA_4,CA,0,0,0,1,...,0,1,1,1,0,2,0,0,0,1


# User-defined functions to calculate Metrics and Croston algorithm

In [12]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [13]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Format sales data and then merge with calendar data

In [14]:
sales_pattern = sales_intermittent.copy()

In [15]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

# Set value for parameters

In [16]:
list_params = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]

In [17]:
list_params

[0.05,
 0.1,
 0.15,
 0.2,
 0.25,
 0.3,
 0.35,
 0.4,
 0.45,
 0.5,
 0.55,
 0.6,
 0.65,
 0.7,
 0.75,
 0.8,
 0.85,
 0.9]

In [18]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

In [19]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [20]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [21]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [22]:
len(sample_list_filtered)

105

# Hyperparameter tuning for Croston

In [23]:
train_data = dataframe_weekly[(dataframe_weekly['wm_yr_wk'] >= start_train_week) & (dataframe_weekly['wm_yr_wk'] <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

In [24]:
start = timer()

df_result_all_params = pd.DataFrame()
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    train_first = train_data_first_part[product]
    train_second = train_data_second_part[product]
    for i in list_params:
        predictions = list()
        history = [x for x in train_first[num_train_needed*-1:]]
        for t in train_second.index.tolist():
            yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(train_second[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                        'Actual Data': train_second,
                                        'Forecast': predictions,
                                        'Alpha': [i for count in range(len(train_second))]})
        df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: FOODS_2_219_CA_1
Progressing: 0.0 %
Currently Running: FOODS_1_157_CA_1
Progressing: 0.95 %
Currently Running: FOODS_3_352_CA_1
Progressing: 1.9 %
Currently Running: FOODS_1_124_CA_4
Progressing: 2.86 %
Currently Running: FOODS_3_183_CA_3
Progressing: 3.81 %
Currently Running: FOODS_3_742_CA_4
Progressing: 4.76 %
Currently Running: FOODS_3_057_CA_1
Progressing: 5.71 %
Currently Running: FOODS_3_614_CA_4
Progressing: 6.67 %
Currently Running: FOODS_3_027_CA_4
Progressing: 7.62 %
Currently Running: FOODS_3_307_CA_1
Progressing: 8.57 %
Currently Running: FOODS_1_091_CA_1
Progressing: 9.52 %
Currently Running: FOODS_3_341_CA_1
Progressing: 10.48 %
Currently Running: FOODS_3_718_CA_1
Progressing: 11.43 %
Currently Running: FOODS_1_022_CA_2
Progressing: 12.38 %
Currently Running: FOODS_3_384_CA_2
Progressing: 13.33 %
Currently Running: FOODS_3_001_CA_3
Progressing: 14.29 %
Currently Running: FOODS_1_192_CA_1
Progressing: 15.24 %
Currently Running: FOODS_3_551_CA_4
Progress

In [25]:
df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,FOODS_2_219_CA_1,3.0,3.331791,0.05
1,FOODS_2_219_CA_1,4.0,3.319284,0.05
2,FOODS_2_219_CA_1,1.0,3.345263,0.05
3,FOODS_2_219_CA_1,13.0,3.254685,0.05
4,FOODS_2_219_CA_1,5.0,3.635397,0.05
...,...,...,...,...
15115,FOODS_3_146_CA_3,0.0,1.712629,0.90
15116,FOODS_3_146_CA_3,3.0,1.712629,0.90
15117,FOODS_3_146_CA_3,3.0,1.529581,0.90
15118,FOODS_3_146_CA_3,0.0,2.722792,0.90


# Summay all metrics

In [26]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [27]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.08859034303333337 minutes


In [28]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_022_CA_2,0.05,0.900424,98.7894,191.6739,97.5091,3.087168,3.975185
1,FOODS_1_022_CA_2,0.10,0.889004,97.5365,183.7914,94.8080,3.048015,3.933097
2,FOODS_1_022_CA_2,0.15,0.867438,95.1704,171.4670,89.6199,2.974074,3.856159
3,FOODS_1_022_CA_2,0.20,0.829095,90.9636,155.1112,80.3319,2.842612,3.725902
4,FOODS_1_022_CA_2,0.25,0.815828,89.5079,148.7191,84.2372,2.797123,3.535316
...,...,...,...,...,...,...,...,...
1885,FOODS_3_784_CA_4,0.70,0.821433,34.6941,36.0682,56.9499,3.989817,4.988491
1886,FOODS_3_784_CA_4,0.75,0.857571,36.2204,37.3117,58.8547,4.165344,5.117938
1887,FOODS_3_784_CA_4,0.80,0.892181,37.6822,38.4948,60.6775,4.333450,5.251412
1888,FOODS_3_784_CA_4,0.85,0.924674,39.0545,39.5989,62.3902,4.491272,5.387462


# Check what products has unexpected metrics result

In [29]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
396,FOODS_2_180_CA_3,0.05,inf,inf,200.0,NaN,0.021277,0.021277
397,FOODS_2_180_CA_3,0.10,inf,inf,200.0,NaN,0.021277,0.021277
398,FOODS_2_180_CA_3,0.15,inf,inf,200.0,NaN,0.021277,0.021277
399,FOODS_2_180_CA_3,0.20,inf,inf,200.0,NaN,0.021277,0.021277
400,FOODS_2_180_CA_3,0.25,inf,inf,200.0,NaN,0.021277,0.021277
401,FOODS_2_180_CA_3,0.30,inf,inf,200.0,NaN,0.021277,0.021277
402,FOODS_2_180_CA_3,0.35,inf,inf,200.0,NaN,0.021277,0.021277
403,FOODS_2_180_CA_3,0.40,inf,inf,200.0,NaN,0.021277,0.021277
404,FOODS_2_180_CA_3,0.45,inf,inf,200.0,NaN,0.021277,0.021277
405,FOODS_2_180_CA_3,0.50,inf,inf,200.0,NaN,0.021277,0.021277


In [30]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
396,FOODS_2_180_CA_3,0.05,inf,inf,200.0,NaN,0.021277,0.021277
397,FOODS_2_180_CA_3,0.10,inf,inf,200.0,NaN,0.021277,0.021277
398,FOODS_2_180_CA_3,0.15,inf,inf,200.0,NaN,0.021277,0.021277
399,FOODS_2_180_CA_3,0.20,inf,inf,200.0,NaN,0.021277,0.021277
400,FOODS_2_180_CA_3,0.25,inf,inf,200.0,NaN,0.021277,0.021277
401,FOODS_2_180_CA_3,0.30,inf,inf,200.0,NaN,0.021277,0.021277
402,FOODS_2_180_CA_3,0.35,inf,inf,200.0,NaN,0.021277,0.021277
403,FOODS_2_180_CA_3,0.40,inf,inf,200.0,NaN,0.021277,0.021277
404,FOODS_2_180_CA_3,0.45,inf,inf,200.0,NaN,0.021277,0.021277
405,FOODS_2_180_CA_3,0.50,inf,inf,200.0,NaN,0.021277,0.021277


In [31]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
504,FOODS_2_221_CA_3,0.05,1.243421,99.4737,NaN,99.6241,1.243421,2.675489
505,FOODS_2_221_CA_3,0.10,1.243421,99.4737,NaN,99.6241,1.243421,2.675489
506,FOODS_2_221_CA_3,0.15,1.243421,99.4737,NaN,99.6241,1.243421,2.675489
507,FOODS_2_221_CA_3,0.20,1.243421,99.4737,NaN,99.6241,1.243421,2.675489
508,FOODS_2_221_CA_3,0.25,1.243421,99.4737,NaN,99.6241,1.243421,2.675489
...,...,...,...,...,...,...,...,...
1471,FOODS_3_444_CA_4,0.70,0.875000,100.0000,NaN,100.0000,1.625000,4.596194
1472,FOODS_3_444_CA_4,0.75,0.875000,100.0000,NaN,100.0000,1.625000,4.596194
1473,FOODS_3_444_CA_4,0.80,0.875000,100.0000,NaN,100.0000,1.625000,4.596194
1474,FOODS_3_444_CA_4,0.85,0.875000,100.0000,NaN,100.0000,1.625000,4.596194


In [32]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
396,FOODS_2_180_CA_3,0.05,inf,inf,200.0,NaN,0.021277,0.021277
397,FOODS_2_180_CA_3,0.10,inf,inf,200.0,NaN,0.021277,0.021277
398,FOODS_2_180_CA_3,0.15,inf,inf,200.0,NaN,0.021277,0.021277
399,FOODS_2_180_CA_3,0.20,inf,inf,200.0,NaN,0.021277,0.021277
400,FOODS_2_180_CA_3,0.25,inf,inf,200.0,NaN,0.021277,0.021277
401,FOODS_2_180_CA_3,0.30,inf,inf,200.0,NaN,0.021277,0.021277
402,FOODS_2_180_CA_3,0.35,inf,inf,200.0,NaN,0.021277,0.021277
403,FOODS_2_180_CA_3,0.40,inf,inf,200.0,NaN,0.021277,0.021277
404,FOODS_2_180_CA_3,0.45,inf,inf,200.0,NaN,0.021277,0.021277
405,FOODS_2_180_CA_3,0.50,inf,inf,200.0,NaN,0.021277,0.021277


In [33]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [34]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [35]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [36]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 1.9047619047619049 %
Percentage of unexpected values of WMAPE is: 1.9047619047619049 %
Percentage of unexpected values of SMAPE is: 5.714285714285714 %
Percentage of unexpected values of MAPE is: 1.9047619047619049 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [37]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [38]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,1.976464,177.552569,113.671541,100.612188,7.079953,8.131501
0.10,1.768883,158.208638,105.274418,92.438794,6.282446,7.330946
0.15,1.623803,145.326310,100.287037,87.644396,5.733450,6.792705
0.20,1.530352,137.055296,97.791187,85.492199,5.360494,6.420529
0.25,1.462098,130.939145,96.262686,84.207997,5.091812,6.152266
0.30,1.411601,126.290281,95.189698,83.411007,4.901978,5.967957
0.35,1.376507,122.879491,94.525528,83.083979,4.770319,5.852420
0.40,1.354125,120.544090,94.301238,83.345027,4.690725,5.785768
0.45,1.338304,118.763698,94.177795,83.638774,4.644039,5.751693


In [39]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.7
The optimum Alpha based on Mean WMAPE is: 0.75
The optimum Alpha based on Mean SMAPE is: 0.5
The optimum Alpha based on Mean MAPE is: 0.35
The optimum Alpha based on Mean MAE is: 0.6
The optimum Alpha based on Mean RMSE is: 0.5


### Get MEDIAN metrics value of each Alpha

In [40]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,1.309016,86.8895,115.6938,71.9423,3.954124,4.607941
0.10,1.111668,75.8770,98.6324,63.9110,3.670187,4.330000
0.15,0.997008,76.1758,99.0938,59.9314,3.589330,4.243416
0.20,0.892849,74.4309,98.0758,59.4701,3.429005,4.185133
0.25,0.847639,74.6550,83.4834,60.6409,3.447667,4.177697
0.30,0.855515,74.9120,82.6296,61.3418,3.566326,4.108989
0.35,0.865497,73.7833,82.8383,62.4621,3.548911,4.066846
0.40,0.874011,73.3134,83.3269,65.0429,3.529703,4.042035
0.45,0.894779,72.5240,83.6190,65.5492,3.506639,4.033434


In [41]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.25
The optimum Alpha based on Median of WMAPE is: 0.75
The optimum Alpha based on Median of SMAPE is: 0.9
The optimum Alpha based on Median of MAPE is: 0.2
The optimum Alpha based on Median of MAE is: 0.85
The optimum Alpha based on Median of RMSE is: 0.45


# Run Croston model after decide best paramaters

In [42]:
best_alpha = 0.35

In [43]:
dataframe_weekly

,wm_yr_wk,FOODS_1_005_CA_2,FOODS_1_005_CA_4,FOODS_1_008_CA_1,FOODS_1_008_CA_4,FOODS_1_009_CA_1,FOODS_1_009_CA_2,FOODS_1_009_CA_3,FOODS_1_009_CA_4,FOODS_1_010_CA_1,...,FOODS_3_821_CA_4,FOODS_3_823_CA_1,FOODS_3_823_CA_2,FOODS_3_823_CA_3,FOODS_3_823_CA_4,FOODS_3_824_CA_1,FOODS_3_824_CA_2,FOODS_3_824_CA_4,FOODS_3_825_CA_4,FOODS_3_826_CA_3
0,11101,36.0,4.0,0.0,1.0,4.0,6.0,6.0,5.0,0.0,...,0.0,0.0,24.0,18.0,2.0,2.0,3.0,1.0,6.0,0.0
1,11102,37.0,13.0,0.0,3.0,5.0,18.0,19.0,5.0,0.0,...,0.0,0.0,30.0,6.0,2.0,1.0,0.0,0.0,0.0,0.0
2,11103,40.0,3.0,0.0,3.0,0.0,8.0,4.0,2.0,0.0,...,0.0,16.0,8.0,31.0,20.0,0.0,0.0,0.0,0.0,0.0
3,11104,30.0,1.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,3.0,20.0,7.0,3.0,1.0,0.0,3.0,0.0
4,11105,33.0,8.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,38.0,6.0,0.0,1.0,2.0,5.0,16.0,6.0,3.0,...,0.0,0.0,17.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0
269,11609,19.0,13.0,0.0,1.0,4.0,18.0,9.0,0.0,0.0,...,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
270,11610,29.0,7.0,0.0,0.0,10.0,3.0,13.0,0.0,2.0,...,0.0,0.0,17.0,0.0,1.0,13.0,7.0,0.0,3.0,0.0
271,11611,43.0,7.0,0.0,2.0,5.0,6.0,7.0,3.0,0.0,...,0.0,0.0,31.0,2.0,2.0,2.0,1.0,1.0,1.0,0.0


In [44]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [45]:
df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    train_product = train_data[product]
    test_product = test_data[product]
    predictions = list()
    history = [x for x in train_product[num_train_needed*-1:]]
    for t in test_product.index.tolist():
        yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
        predictions.append(yhat)
        history.append(test_product[t])
    df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                    'Actual Data': test_product,
                                    'Forecast': predictions})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)

In [46]:
df_result_best_params

,Product,Actual Data,Forecast
0,FOODS_1_008_CA_1,0.0,1.174615
1,FOODS_1_008_CA_1,0.0,1.174615
2,FOODS_1_009_CA_1,5.0,5.983680
3,FOODS_1_009_CA_1,7.0,5.640114
4,FOODS_1_010_CA_1,0.0,0.981025
...,...,...,...
2597,FOODS_3_823_CA_4,0.0,1.668804
2598,FOODS_3_824_CA_4,1.0,1.008723
2599,FOODS_3_824_CA_4,0.0,0.486184
2600,FOODS_3_825_CA_4,1.0,4.427773


In [47]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [48]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_005_CA_2,0.638647,41.2664,42.8502,42.2404,13.411579,13.849563
1,FOODS_1_005_CA_4,0.506787,22.5239,22.6397,18.8549,2.027148,2.753211
2,FOODS_1_008_CA_1,inf,inf,200.0000,NaN,1.174615,1.174615
3,FOODS_1_008_CA_4,0.801570,68.7060,92.9479,61.3091,2.404710,2.846814
4,FOODS_1_009_CA_1,0.585891,19.5297,19.7143,19.5503,1.171783,1.186785
...,...,...,...,...,...,...,...
1296,FOODS_3_824_CA_1,0.508304,101.6608,106.7561,12.6570,1.016608,1.271368
1297,FOODS_3_824_CA_2,0.502486,100.4973,100.1324,0.2651,0.502486,0.708751
1298,FOODS_3_824_CA_4,0.247453,49.4906,100.4342,0.8723,0.247453,0.343839
1299,FOODS_3_825_CA_4,inf,283.1001,115.9184,283.1001,2.831001,2.893217


In [49]:
df_result_final.isnull().sum()

Product      0
MASE       333
WMAPE      333
SMAPE      333
MAPE       605
MAE          0
RMSE         0
dtype: int64

In [50]:
df_result_final.MAPE.mean()

68.93336968390804

In [51]:
df_result_final.to_csv('Croston_Intermittent_Metrics.csv')